In [2]:
import requests
import time
import json
import traceback
import datetime
import logging
import base64
import re
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

session = requests.session()

# 请求头
headers = {
    'Upgrade-Insecure-Requests': '1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
}
'''
请求中心，控制程序所有HTTP请求，如果请求发生错误进行尝试再次连接
@param url 请求连接
@return 请求响应结果
'''
def fofa_requests(url):
    # print(url)
    rs_content = ''
    count = 3
    while count != 0:
        try:
            rs = session.get(url, verify=False,headers=headers)
            rs_text = rs.text
            results = json.loads(rs_text)
            print(results)
            #total_size = results['size']
            error = results
            if results['error'] and 'None' not in results['error']:
                info = u'fofa 错误:'+results['error']+u' 休眠10s'
                print(info)
                time.sleep(10)
                count += -1
            else:
                rs_content = results
                break
            
        except Exception as e:
            # logging.error(u'fofa 错误:'+str(e.message)+u' 休眠10s')
            print('exception', e)
            time.sleep(10)
            count += -1
    return rs_content


def parse_result(fields, results, query, source):
    list_result = []
    list_fields = str(fields).split(',')
    len_fields = len(list_fields)
    for result in results:
        dic_row = {}
        for index in range(len_fields):
            dic_row[list_fields[index]] = result[index]
        dic_row['query_date'] =  re.search("\r\nDate: (.*?)\r\n", dic_row["header"]).group(1)
        dic_row['query_date'] = str(datetime.datetime.strptime(dic_row['query_date'], '%a, %d %b %Y %H:%M:%S GMT'))
        dic_row['query'] = query
        dic_row['source'] = source
        if "header" in dic_row.keys():
            del dic_row["header"]
        list_result.append(dic_row) 
        
    return list_result
        
'''
@param fofa_sql fofa查询语句
'''
def fofa_query(rule, size, time_select=True):
    query_str = add_time(rule, 1) if time_select ==  True else rule
    print(query_str)
    list_result = []
    # 参数定义
    #email = '.com'
    email = '@sm'
    # key = ''
    key = ''
    qbase64 = base64.b64encode(query_str.encode('utf-8')).decode('utf-8')
    fields = 'host,title,ip,domain,port,country,city,header,server,as_number,as_organization'
    page = 1
    api_url = 'http://fofa.so/api/v1/search/all?email={}&key={}&fields={}&page={}&size={}&qbase64={}&full=true'.format(email,key,fields,page,size,qbase64)
    print(api_url)
    # 获取第一次结果
    rs = fofa_requests(api_url)
    if rs == '':
        list_result
    total_size = rs['size']

    # 计算页数
    page_end = total_size / size + 1 if total_size > size else page

    # 获取结果
    list_result.extend(parse_result(fields, rs['results'], rule, 'fofa'))
    for page_no in range(page + 1 ,int(page_end)+1):
        api_url = 'http://fofa.so/api/v1/search/all?email={}&key={}&fields={}&page={}&size={}&qbase64={}&full=true'.format(email,key,fields,page_no,size,qbase64)
        rs = fofa_requests(api_url)
        list_result.extend(parse_result(fields, rs['results'], rule, 'fofa'))
    
    return list_result


def add_time(query_str, daysago):
    after = datetime.date.today()- datetime.timedelta(days=daysago)
    return query_str + ' && after="{}"'.format(after)

if __name__ == '__main__':
    #print(fofa_query('fid="G6XS9/fV1xXt51WgFSVkzA=="', False))
    rulers_file = 'rulers.txt'
    with open(rulers_file, 'r') as f:
        for line in f:
            rule,size = line.strip().split(',')
            print(base64.b64encode(rule.encode('utf-8')).decode('utf-8'))
            #res = fofa_query(rule, int(size), False)
            break
# Ym9keT0iZmlsZUlkID0gNDg1NzMzMTk1NjAyIg==    

Ym9keT0iZmlsZUlkID0gNDg1NzMzMTk1NjAyIg==


In [3]:
import requests
headers = {
    'Upgrade-Insecure-Requests': '1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
}
url = 'http://fofa.info/api/v1/search/all?email=m631141639@sina.com&key=b3fa5d1490af3df220dd7ec73d8e3dad&fields=host,title,ip,domain,port,country,city,header,server,as_number,as_organization&page=1&size=500&qbase64=Ym9keT0ibWltZy4xMjcubmV0L3AvaW1hZ2VzL2Zhdmljb24uaWNvIg==&full=true'
res = requests.get(url, verify=False,headers=headers)

In [4]:
import json
for key, value in json.loads(res.text).items():
    if key == 'results':
        for v in value:
            print(v[0])

136.244.94.12
tangfen.qghbgs163.com
https://136.244.94.12
https://108.61.209.242
jewban.jiuban126.com
104.168.194.148
108.61.209.242
https://209.250.238.130
jsdab.houdian126.com
192.248.151.84
209.250.238.130
fgwtzk.xqswj126.com
https://192.248.151.84
https://149.248.9.142
cc188aevesconwload.com
www.cc188aevesconwload.com
149.248.9.142
95.179.213.49
104.168.194.148:443
zfghk.jiuban126.com
https://95.179.213.49
https://ftp.cc188aevesconwload.com
ftp.cc188aevesconwload.com
https://95.179.233.96
fenju.xqswj126.com
95.179.233.96
wtzbghk.wangcom.company
https://108.61.176.243
https://149.248.45.107
149.248.45.107
163mailcommail.com
www.163mailcommail.com
https://45.76.111.201
foxmailfree.com
www.foxmailfree.com
45.32.228.244
fgwjing.zjbswjs.com
45.76.111.201
https://216.128.144.188
163mailcommaildate.com
www.163mailcommaildate.com
216.128.144.188
https://45.32.228.244
https://45.76.212.26
https://216.238.100.175
gxqxinxiangmail.com
70.34.249.65
216.238.100.175
motgovcn.vip
www.motgovcn.vip
